# CNNの可視化 (Gradient-weighted Class Activation Mapping; Grad-CAM)

---

## 目的
Gradient-weighted Class Activation Mapping (Grad-CAM)の仕組みを理解する.

Grad-CAMを用いてCIFAR-10データセットに対するネットワークの判断根拠の可視化を行う．

## Gradient-weighted Class Activation Mapping (Grad-CAM)
Grad-CAM[1]は，逆伝播時の正値の勾配を用いることでCNNを可視化する手法です．
Grad-CAMは，逆伝播時の特定のクラスにおける勾配をGlobal Average Pooling (GAP)[2]により空間方向に対する平均値を求め，各特徴マップに対する重みとします．
その後，獲得した重みを各特徴マップに重み付けすることでAttention map を獲得します．
02_CAM.ipynbで使用したClass Activation Mapping (CAM)[3]は，ネットワークの一部をGAPに置き換える必要があるため，Attention mapを獲得するためにネットワークを学習させる必要があります．一方で，Grad-CAMはネットワークの順伝播時の特徴マップと逆伝播時の勾配を用いてAttention mapを獲得します．そのため，学習済みの様々なネットワークからAttention map を獲得することができます．


<img src="https://www.dropbox.com/s/x23sm70ftoo7caa/grad-cam.png?dl=1" width = 100%>


## pytorch-gradcamのインストール

Grad-CAMを利用するために必要なツールをインストールします．
Grad-CAMは，`pytorch-gradcam`というツールをインストールすることで簡単に利用することができます．

In [ ]:
!pip install pytorch-gradcam

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．
今回はPyTorchのライブラリに加えて，
上でインストールしたpytorch-gradcam (`gradcam`) もインポートします．

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models

import torchsummary

from gradcam import GradCAM
from gradcam.utils import visualize_cam

## GPUの確認
GPUを使用した計算が可能かどうかを確認します．
下記のコードを実行してGPU情報を確認します． GPUの確認を行うためには，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．

`Use CUDA: True`と表示されれば，GPUを使用した計算をPytorchで行うことが可能です． CPUとなっている場合は，上記に記載している手順にしたがって，設定を変更してください．

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Use Device:', device)

## 使用するデータセット

### データセット
今回の物体認識では，CIFAR-10データセットを使用します．CIFAR-10データセットは，飛行機や犬などの10クラスの物体が表示されている画像から構成されたデータセットです．

![CIFAR10_sample.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/176458/b6b43478-c85f-9211-7bc6-227d9b387af5.png)

### データセットのダウンロードと読み込み
実験に使用するCIFAR-10データセットを読み込みます．
１回の誤差を算出するデータ数 (ミニバッチサイズ) は，64とします．
まず，CIFAR-10データセットをダウンロードします．
次に，ダウンロードしたデータセットを読み込みます．
学習には，大量のデータを利用しますが，それでも十分ではありません． そこで，データ拡張 (data augmentation) により，データのバリエーションを増やします． 一般的な方法は，画像の左右反転，明るさ変換などです．

In [ ]:
batch_size = 64

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=20)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=20)

classes_list = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

## ネットワークモデルの定義
学習済みモデルにはResNet-18を利用して学習します．`pretrained = True`にすると，ImageNetで学習したモデルを利用できます．ここで，ImageNetは1,000クラスのデータセットです．すなわち，ImageNetで学習したResNet-18の出力層のユニット数は1,000になっています．ファインチューニングに利用するCIFAR-10データセットは10クラスなので，出力層のユニット数を変更します．


## 損失関数と最適化手法の定義
学習に使用する損失関数と最適化手法を定義します．
各更新において，学習用データと教師データをそれぞれ`inputs`と`targets`とします．
学習モデルに`inputs`を与えて，ResNetの出力を取得します．
ResNetの出力と教師ラベル`targets`との誤差を`criterion`で算出します．
また，認識精度も算出します．
そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．
最適化手法には，確率的勾配降下法 (stochastic gradient descent: SGD) を用いて学習します．

最後に，定義したネットワークの詳細情報を`torchsummary.summary()`関数を用いて表示します．

In [ ]:
# ネットワークモデルの定義
model = models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# 損失関数と最適化手法の定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# モデルと損失関数をGPU演算へ移動
model.to(device)
criterion.to(device)

# モデルの情報を表示
torchsummary.summary(model, (3, 32, 32))

## 学習とテスト
学習エポック数を3とします．
CIFAR-10データセットの学習データサイズを取得し，1エポック内における更新回数を求めます．
1エポック学習するごとに学習したモデルを評価し，最も精度の高いモデルが保存されます．

In [ ]:
epochs = 3
best_acc = 0  # best test accuracy

for epoch in range(epochs):
    # training
    model.train()
    train_loss, train_running_acc = 0.0, 0.0
    correct, total, count = 0, 0, 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # print statistics
        train_running_acc += 100.*correct/total
        count += 1

    print('[Epoch %d] Train Loss: %.5f | Train Acc: %.3f%%'
                  % (epoch + 1, train_loss/count, train_running_acc/count))

    # testing
    model.eval() 
    with torch.no_grad():
        test_loss, test_running_acc = 0.0, 0.0
        correct, total, count = 0, 0, 0
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # print statistics
            test_running_acc += 100.*correct/total
            count += 1

        print('Test Loss: %.5f | Test Acc: %.3f%%'
                      % (test_loss/count, test_running_acc/count))

    # save model
    if test_running_acc/count > best_acc:
        best_acc = max(test_running_acc/count, best_acc)
        torch.save(model.state_dict(), './cifar_net.pth')
        print("Save best model ...")
    
print('Finished Training')
print("Best Test Accuracy: %.3f%%" % best_acc)

## 学習済みモデルの読み込み

Grad-CAMによる可視化を行います．

まずはじめに，上の学習で最も精度の良かった時のネットワークモデルを用意します．

上の学習で得られた最高精度のネットワークモデルのパラメータを読み込みます．
まず，`torch.load()`で指定したファイルを読み込み，ファイル内のデータを返します．
そして読み込んだデータを`model.load_state_dict()`で`model`へ読み込むことで，保存したネットワークパラメータに沖狩ることができます．

In [ ]:
model.load_state_dict(torch.load('./cifar_net.pth'))

##Grad-CAMによるAttention mapの獲得

Grad-CAMによりAttention mapを可視化して，ネットワークの判断根拠を確認してみます． 再度，実行することで他のテストサンプルに対するAttention mapを可視化することができます． pred (prediction) は認識結果，conf (confidence) は認識結果に対する信頼度を示しています．


In [ ]:
# Grad-CAMで可視化する層の指定
# (ex., layer1, layer2, layer3, layer3[1], layer4[0])
target_layer = model.layer4

# Grad-CAMを計算するためのモデルの準備
gradcam = GradCAM(model, target_layer)

# テストデータを入力し，認識結果を獲得する
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = model(inputs)

    conf_data = outputs.data.topk(k=1, dim=1, largest=True, sorted=True)
    _, predicted = outputs.max(1)
    d_inputs = inputs.data.cpu().numpy()
    break

# 得られたAttention mapデータを保存するためのリストを用意
v_list = []
att_list = []

# 上の認識に使用した画像データを1枚ずつ読み込んでGrad-CAMを計算
for i in range(inputs.shape[0]):
    # 画像データを1枚選択
    input = inputs[i:i+1,:,:,:]

    mask, _ = gradcam(input)
    _, result = visualize_cam(mask, input)

    v_img = d_inputs[i,:,:,:]
    v_img = v_img.transpose(1, 2, 0) * 255
    v_img = np.uint8(v_img)
    v_list.append(v_img)

    result = result * 255.0
    result = np.uint8(result)
    result = result.transpose((1, 2, 0))
    att_list.append(result)

## Attention mapの表示

上のプログラムで獲得したAttention mapをmatplotlibを用いて表示します．

In [ ]:
# 行・列の数を指定
cols, rows = 8, 1

fig = plt.figure(figsize=(14, 3.0))
plt.title('Input image')
plt.axis("off")
for r in range(rows):
    for c in range(cols):
        cls = targets[c].item()
        ax = fig.add_subplot(r+1, cols, c+1)
        plt.title('{}'.format(classes_list[cls]))
        ax.imshow(v_list[cols * r + c])
        ax.set_axis_off()
plt.show()

fig = plt.figure(figsize=(14, 3.5))
plt.title('Attention map')
plt.axis("off")
for r in range(rows):
    for c in range(cols):
        pred = predicted[c].item()
        conf = conf_data[0][c].item()
        ax = fig.add_subplot(r+1, cols, c+1)
        ax.imshow(att_list[cols * r + c])
        plt.title('pred: {}\nconf: {:.2f}'.format(classes_list[pred], conf))
        ax.set_axis_off()
plt.show()

#課題
1. Attention mapを可視化する層を変更して，Attention mapの変化を確認してみましょう．


# 参考文献
- [1] S. Ramprasaath, R., C. Michael, D. Abhishek,
V. Ramakrishna, P. Devi, and B.
Dhruv, "Grad-CAM: Visual explanations from deep networks
via gradient-based localization". In International Conference
on Computer Vision, pp. 618–626, 2017.

- [2] B. Zhou, A. Khosla, A. Lapedriza, A. Oliva,
and A. Torralba, "Learning deep features for discriminative
localization". In 2016 IEEE Conference on Computer
Vision and Pattern Recognition, pp. 2921–2929, 2016.

- [2] M. Lin, Q. Chen, and S. Yan, "Network in network".
In 2nd International Conference on Learning Representations,
Banff, AB, Canada, April 14-16, 2014, Conference
Track Proceedings, 2014.